# **라이브러리**

In [100]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import *
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

import warnings
warnings.filterwarnings(action='ignore')

In [45]:
dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\Ubion-CorporateBankruptcyAnalysis\Data\최종데이터셋\Finaldataset_ver7.csv')

In [46]:
dataset.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '상장폐지일', '감사의견코드', '자기자본배당률',
       '금융비용 대 부채비율', '자기자본배율', '유동부채비율', '매입채무회전률', '유동자산회전률', '총자본투자효율',
       '순운전자본비율', '누적수익성비율', '총자산영업이익률', '총자산회전율', '년', 'key', '부도', '공시발생일',
       '정기공시제목', '수시공시제목', '대표이사 변경', '최대주주 변경', '회계처리위반', '횡령배임', '폐지사유요약합본',
       '신종채권'],
      dtype='object')

In [47]:
drop_feature = ['회사명', '거래소코드', '회계년도', '소속코드', '상장폐지일', '감사의견코드', '년', 'key','공시발생일', '정기공시제목', '수시공시제목','폐지사유요약합본', '부도']
text_col = ['대표이사 변경', '최대주주 변경', '회계처리위반', '횡령배임','신종채권']

In [167]:
ajX.corr()

,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,매입채무회전률,유동자산회전률,총자본투자효율,순운전자본비율,누적수익성비율,총자산영업이익률,총자산회전율,대표이사 변경,최대주주 변경,회계처리위반,횡령배임,신종채권
자기자본배당률,1.000000,-0.020700,-0.000570,-0.003388,0.000262,0.007070,0.010604,0.021709,0.021878,0.018576,0.016965,-0.011808,-0.010926,0.000025,-0.004019,-0.014904
금융비용 대 부채비율,-0.020700,1.000000,-0.002963,0.026849,-0.005203,-0.041732,-0.100294,-0.178712,-0.257706,-0.148333,-0.095196,0.167698,0.161005,0.027280,0.072014,0.198657
자기자본배율,-0.000570,-0.002963,1.000000,-0.002079,-0.000014,-0.017098,-0.004975,0.028683,-0.010584,-0.005630,-0.018408,-0.001313,-0.005008,-0.002124,-0.001978,-0.006233
유동부채비율,-0.003388,0.026849,-0.002079,1.000000,-0.001720,0.014019,-0.023679,-0.089369,-0.046215,-0.040408,0.013819,0.009121,0.007590,0.002342,0.006874,0.006394
매입채무회전률,0.000262,-0.005203,-0.000014,-0.001720,1.000000,0.066314,0.004621,0.001570,0.004255,0.004253,0.028498,-0.002407,-0.002632,-0.002130,-0.001310,-0.004811
유동자산회전률,0.007070,-0.041732,-0.017098,0.014019,0.066314,1.000000,0.108279,-0.259727,0.085256,0.119613,0.605795,-0.072685,-0.079709,0.001486,-0.028796,-0.101919
총자본투자효율,0.010604,-0.100294,-0.004975,-0.023679,0.004621,0.108279,1.000000,0.316574,0.526173,0.556852,0.064893,-0.159148,-0.121416,-0.027731,-0.075026,-0.088451
순운전자본비율,0.021709,-0.178712,0.028683,-0.089369,0.001570,-0.259727,0.316574,1.000000,0.487746,0.386045,-0.084226,-0.119575,-0.081410,-0.039519,-0.060288,-0.103574
누적수익성비율,0.021878,-0.257706,-0.010584,-0.046215,0.004255,0.085256,0.526173,0.487746,1.000000,0.642769,0.023528,-0.254571,-0.203486,-0.067589,-0.152911,-0.164466
총자산영업이익률,0.018576,-0.148333,-0.005630,-0.040408,0.004253,0.119613,0.556852,0.386045,0.642769,1.000000,0.073868,-0.196869,-0.122579,-0.026376,-0.059269,-0.104060


In [168]:
ajX = dataset.drop(drop_feature, axis=1)
X = ajX.drop(text_col, axis = 1)
ajy = dataset['부도']
y = dataset['부도']

In [169]:
X, y = RandomUnderSampler(random_state=0).fit_resample(X, y)

ajX , ajy = RandomUnderSampler(random_state=0).fit_resample(ajX , ajy)

In [170]:
def modeling(model):    
    model.fit(X_train, y_train)
    pred = model.predict_proba(X_test)
    pred = (pred[:,0] < 0.1)*1         # 부도기업일 확률이 10% 이상이면 부도로 판단해라.
    return pred

# **재무데이터 + 비재무데이터**

In [171]:
X_train, X_test, y_train, y_test = train_test_split(ajX, ajy, test_size=0.3, stratify=ajy, random_state = 46)

In [172]:
dt_clf = modeling(DecisionTreeClassifier())
rf_clf = modeling(RandomForestClassifier())
ada_clf = modeling(AdaBoostClassifier())
lg_clf = modeling(LogisticRegression())
knn_clf = modeling(KNeighborsClassifier())
svc_clf = modeling(SVC(probability=True))
lgmb_clf = modeling(LGBMClassifier())

In [176]:
confusion_matrix(y_test, lgmb_clf)  # 59 62

array([[112,   9],
       [ 32,  89]], dtype=int64)

# **재무데이터only**

In [174]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state = 46)

ajdt_clf = modeling(DecisionTreeClassifier())
ajrf_clf = modeling(RandomForestClassifier())
ajada_clf = modeling(AdaBoostClassifier())
ajlg_clf = modeling(LogisticRegression())
ajknn_clf = modeling(KNeighborsClassifier())
ajsvc_clf = modeling(SVC(probability=True))
ajlgmb_clf = modeling(LGBMClassifier())

In [177]:
confusion_matrix(y_test, ajlgmb_clf) # 42 79

array([[107,  14],
       [ 37,  84]], dtype=int64)